# Start code

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))


cuda
0
NVIDIA GeForce RTX 4070


# Create Torch Dataset

In [3]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)
            # self.data is a list of dicts with kps and img associations from the json
            
        #transformations used later to process the numpy array from using cv2.imread() on the images
        self.transforms = transforms.Compose([
            #converts from numpy array to PIL
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            #converts to a tensor for use with Pytorch 
            transforms.ToTensor(),
            #normalizes image for use with imagenet models
            transforms.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        """gets an item from the datatset after normalizing the images and keypoints for pytorch"""
        item = self.data[index]
        img = cv2.imread(f'{self.img_dir}/{item['id']}.png')
        h, w = img.shape[:2]
        
        #BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        #converts the keypoints from [x,y],[x,y] to [x,y,x,y]
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)
        
        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates
        
        return img, kps
    
    

        

In [4]:
train_dataset = KeypointsDataset('data/images', 'data/data_train.json')
val_dataset = KeypointsDataset('data/images', 'data/data_val.json')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle = False)

# Create Model

In [19]:
model = models.resnet50(pretrained=True) #resnet50 is used for image classificaiton
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) #replaces last layer to output 28 numbers, 14 sets of x,y


In [20]:
model = model.to(device)

# Training Model

In [21]:
#the loss between predicted kps and actual kps
criterion = torch.nn.MSELoss()
#optimizer updates model, its rate is 1e^-4
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [22]:
#each epoch is a pass through the dataset
num_epochs = 20
for epoch in range(num_epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)
        
        #empties gradients
        optimizer.zero_grad()
        #passes batch of imgs to model
        outputs = model(imgs)
        #calculates loss
        loss = criterion(outputs, kps)
        #calculates the gradients or how off each weight/parameter is via backpropagation
        loss.backward()
        #updates models parameters in direction opposite to gradient
        optimizer.step()
        
        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Step {i+1}/{len(train_loader)}, Loss {loss.item()}")

Epoch 1/20, Step 1/829, Loss 14449.73046875
Epoch 1/20, Step 11/829, Loss 14775.2861328125
Epoch 1/20, Step 21/829, Loss 13907.263671875
Epoch 1/20, Step 31/829, Loss 14195.8046875
Epoch 1/20, Step 41/829, Loss 13685.951171875
Epoch 1/20, Step 51/829, Loss 12878.169921875
Epoch 1/20, Step 61/829, Loss 12612.4736328125
Epoch 1/20, Step 71/829, Loss 12092.6748046875
Epoch 1/20, Step 81/829, Loss 11979.791015625
Epoch 1/20, Step 91/829, Loss 11849.1259765625
Epoch 1/20, Step 101/829, Loss 12344.5537109375
Epoch 1/20, Step 111/829, Loss 10566.830078125
Epoch 1/20, Step 121/829, Loss 10513.77734375
Epoch 1/20, Step 131/829, Loss 9477.615234375
Epoch 1/20, Step 141/829, Loss 10050.7998046875
Epoch 1/20, Step 151/829, Loss 9305.7861328125
Epoch 1/20, Step 161/829, Loss 9064.08203125
Epoch 1/20, Step 171/829, Loss 8698.423828125
Epoch 1/20, Step 181/829, Loss 8303.0068359375
Epoch 1/20, Step 191/829, Loss 8059.69921875
Epoch 1/20, Step 201/829, Loss 7473.189453125
Epoch 1/20, Step 211/829, Los

In [24]:
torch.save(model.state_dict(), "keypoints_model.pth")